In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/minsir/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "minsirx.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from apps.email_manager.service_layer.insurance_erp_llm import InsuranceERPLLM

/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [2]:
insLLM = InsuranceERPLLM()
insLLM.store_documents_in_vector_store(["/Users/mirbilal/Desktop/minsir/media/email_attachments/PL-0123-202504-E12-000001_-_LAPTOPS_5PTg6tE.pdf"])
# insLLM.label_documents()
insLLM.search_documents_vector_store()
insLLM.query_from_final_results(insLLM.document_query_results)
# insLLM.clean_query_results()
print("----")
print(insLLM.queries_results)

/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in

{'net_premium': {'Choice 1': 'LMA5394 - Communicable diseases clause\n504 Cover of Mobile and Portable Equipment\n600 Maintenance Clause1: Business Type\nPremium Computation ANYWHERE IN PAKISTANSection Material Damage\nLAPTOPS (DETAIL AS PER LIST ATTACHED)              790,499Sum Insured (Rs) Description of Items, Make & Manufacturing Year1:\nLocation.\nSPECIFICATION OF THE PROPERTY INSURED\nDeductible (Rs) Each & Every Loss\n10% of loss amount against Partial\nloss or 10% of sum insured in case\nof total loss\n790,499', 'Choice 2': 'Geographical Area :Pakistan1ST FLOOR, 141-CCA, PHASE - IV, DHA, LAHORE  Address :\nGROSS PREMIUM\nASC\nSUB TOTAL\nFIF\nSales Tax On Services/F.E.D.\nSTAMP DUTY\nNET PREMIUM 23,715\n1,186\n24,901\n249\n3,237\n50\n28,4375\n1\n13Total Sum Insured: Place Of Issue\nENDORSEMENT attachment to this Policy ----- Title and No.\nSUBJECT TO :\n--- Subject to no known or reported loss till date 13-01-2023.\nLMA5394 - Communicable diseases clause\n504 Cover of Mobile an

In [4]:
from datetime import datetime
import re


for key, value_dict in insLLM.queries_results.items():
    data_type = value_dict["type"]
    data_value = value_dict["value"]
    # value_dict["value"] = None
    # continue
    if data_value is not None and type(data_value) is str and "404 Not Found" not in data_value:
        # Clean the data value
        data_values = data_value.split("=")
        cleaned_value = data_values[-1].strip().replace("'", "")

        # Convert the cleaned data value to the specified type
        if data_type == "double":
            try:
                # Extract numbers from the string using regex
                cleaned_value = re.sub(r'[^\d.]', '', cleaned_value)
                cleaned_value = float(cleaned_value)
            except ValueError:
                cleaned_value = None
        elif data_type == "datetime":
            try:
                # Extract date in the format YYYY-MM-DD using regex
                match = re.search(r'\d{4}-\d{2}-\d{2}', cleaned_value)
                if match:
                    cleaned_value = datetime.strptime(match.group(), "%Y-%m-%d")
                else:
                    cleaned_value = None
            except ValueError:
                cleaned_value = None
        elif data_type == "string":
            cleaned_value = str(cleaned_value)
        
        # Update the value in the dictionary
        value_dict["value"] = cleaned_value

print(insLLM.queries_results)

{'net_premium': {'value': '404 Not Found', 'type': 'double'}, 'issue_date': {'value': datetime.datetime(2024, 2, 23, 0, 0), 'type': 'datetime'}, 'covernote_number': {'value': None, 'type': 'string'}, 'policy_number': {'value': 'PL-0223-202504-V01-000212', 'type': 'string'}, 'claim_number': {'value': None, 'type': 'string'}, 'customer_name': {'value': 'PLASCOM MOULDING SERVICES', 'type': 'string'}, 'risk_type': {'value': 'Auto Insurance', 'type': 'string'}, 'event_type': {'value': 'policy_issued', 'type': 'string'}, 'sum_insured': {'value': 2800000.0, 'type': 'double'}, 'premium_paid_amount': {'value': None, 'type': 'double'}, 'claim_intimation_amount': {'value': None, 'type': 'double'}, 'claim_paid_amount': {'value': None, 'type': 'double'}, 'premium_paid_date': {'value': None, 'type': 'datetime'}, 'claim_intimation_date': {'value': None, 'type': 'datetime'}, 'claim_paid_date': {'value': None, 'type': 'datetime'}}


In [25]:
insLLM.label_documents()

In [26]:
print(insLLM.doc_label)

claim_intimated


In [3]:
dummy_convo = [
    {
        "from": "john.doe@example.com",
        "to": ["support@insurancecompany.com"],
        "date": "June 15, 2023, 10:30 AM",
        "subject": "Inquiry about Insurance Details",
        "body": """
Dear Support Team,

I have a few queries regarding my insurance policy:
1. What is the net premium amount?
2. What is the starting date of the coverage period?
3. What is the cover note number which starts with 'COV'?
4. What is the policy number which starts with 'PL'?

Thank you for your assistance.

Best regards,
John Doe
"""
    },
    {
        "from": "support@insurancecompany.com",
        "to": ["john.doe@example.com"],
        "date": "June 15, 2023, 1:45 PM",
        "subject": "Re: Inquiry about Insurance Details",
        "body": """
Dear Mr. Doe,

Thank you for reaching out to us. Please find the answers to your queries below:
1. The net premium amount for your policy is $1,200.
2. The starting date of the coverage period is January 1, 2023.
3. The cover note number is COV123456.
4. The policy number is PL987654.

If you have any further questions, please do not hesitate to contact us.

Best regards,
Insurance Support Team
"""
    },
    {
        "from": "john.doe@example.com",
        "to": ["support@insurancecompany.com"],
        "date": "June 16, 2023, 9:00 AM",
        "subject": "Re: Inquiry about Insurance Details",
        "body": """
Dear Support Team,

Thank you for the information. Could you please confirm if the cover note number COV123456 is correct? I have a document that mentions a different cover note number.

Best regards,
John Doe
"""
    },
    {
        "from": "support@insurancecompany.com",
        "to": ["john.doe@example.com"],
        "date": "June 16, 2023, 11:15 AM",
        "subject": "Re: Inquiry about Insurance Details",
        "body": """
Dear Mr. Doe,

We apologize for any confusion. After rechecking our records, the correct cover note number is COV654321.

Thank you for your patience.

Best regards,
Insurance Support Team
"""
    }
]

In [4]:
insLLM = InsuranceERPLLM()
insLLM.store_convo_in_vector_store(f"{dummy_convo}")
insLLM.search_convo_vector_store()
insLLM.query_from_final_results(insLLM.convo_query_results)

{
    "net_premium": {
        "Choice 1": "the policy number which starts with 'PL'?\\n\\nThank you for your assistance.\\n\\nBest regards,\\nJohn Doe\\n\"}, {'from': 'support@insurancecompany.com', 'to': ['john.doe@example.com'], 'date': 'June 15, 2023, 1:45 PM', 'subject': 'Re: Inquiry about Insurance Details', 'body': '\\nDear Mr. Doe,\\n\\nThank you for reaching out to us. Please find the answers to your queries below:\\n1. The net premium amount for your policy is $1,200.\\n2. The starting date of the coverage period is January 1, 2023.\\n3. The",
        "Choice 2": "for your policy is $1,200.\\n2. The starting date of the coverage period is January 1, 2023.\\n3. The cover note number is COV123456.\\n4. The policy number is PL987654.\\n\\nIf you have any further questions, please do not hesitate to contact us.\\n\\nBest regards,\\nInsurance Support Team\\n'}, {'from': 'john.doe@example.com', 'to': ['support@insurancecompany.com'], 'date': 'June 16, 2023, 9:00 AM', 'subject': 'Re

/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the en

=event type=policy_issued
policy_issued
-----
net_premium =1,200
-----
customer_name =PL987654
-----
risk_type =Commercial Property Insurance
-----
sum_insured = $1,200
-----


In [58]:
print(insLLM.get_vectors(insLLM.documents_vector_store)[0].shape)

(768,)
